# README.md


# approach #3; just use the Medtable info from Sean's table

In [2]:
## Research question: For the approach #2 big cohort, how many have ADT window assgined, 
## with T baseline, and have T recovery and non-recovery status? 

## Table of Contents (Feb23)

# clean up data; fix dates, get stats
# get ADT windows
# T baseline avail
# T recovery non recovery 
#input files-covari-plot TR curves

# Step1, read from spreadsheet, clean up, and get statistics

In [7]:
import pandas as pd
from pandas import DataFrame

import numpy as np

import sys, getopt
import csv
from datetime import datetime, timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

In [8]:
df1=pd.read_excel("./data/sean_table_feb10_approch3_just_from_medtable.xlsx", header=0)
df1.replace(r'', np.NaN)
#df1.fillna('', inplace=True)
#df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])

df_test=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)
df_test.replace(r'', np.NaN)
df_test[:3]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.0
1,2,2,2003,1.0,7.0,2003-01-07,2007-01-09,49041-7,TESTOSTERONE,304.0
2,2,2,2003,1.0,7.0,2003-01-07,2007-05-17,49041-7,TESTOSTERONE,259.0


In [9]:
print(shape(df1))
print(shape(df_test))

(12250, 12)
(18643, 10)


In [10]:
df1[20:25]

,person_id,dxdate,dx_year,source_from,evidence,hormone_therapy,cancer_drug,matched_text,category,date_year,date_month,date_day
20,13,2007-03-15,2007,Medtable,GOSERELIN 10.8 MG SUBQ IMPLANT,ADT,Goserelin,goserelin,Hormone Therapy,2010.0,12.0,16.0
21,13,2007-03-15,2007,Medtable,GOSERELIN 10.8 MG SUBQ IMPLANT,ADT,Goserelin,goserelin,Hormone Therapy,2011.0,3.0,24.0
22,13,2007-03-15,2007,Medtable,GOSERELIN 10.8 MG SUBQ IMPLANT,ADT,Goserelin,goserelin,Hormone Therapy,2011.0,8.0,12.0
23,13,2007-03-15,2007,Medtable,GOSERELIN 10.8 MG SUBQ IMPLANT,ADT,Goserelin,goserelin,Hormone Therapy,2011.0,11.0,10.0
24,13,2007-03-15,2007,Medtable,GOSERELIN 10.8 MG SUBQ IMPLANT,ADT,Goserelin,goserelin,Hormone Therapy,2012.0,2.0,2.0


In [11]:
df1_pt_grouped=df1.groupby(["person_id"]).size().reset_index(name='counts')
list_pt=df1_pt_grouped["person_id"].to_list()
print("")
print("Total number of pts")
print(len(list_pt))


Total number of pts
2120


In [12]:
list_pt[:10]

[2, 12, 13, 18, 22, 25, 26, 37, 38, 53]

In [13]:
###make sure all pts are included in the T test result table

df_tvalue_grouped=df_test.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_test=df_tvalue_grouped["person_id"].to_list()
print("list of pt ids with t test results")
print(len(list_pt_test))
print("")
print(len(list_pt))

overlap_list=[x for x in list_pt if x in  list_pt_test]
print(len(overlap_list))

list of pt ids with t test results
4490

2120
1099


In [14]:
#get the subtable with date_month empty
#get the subtable with data_month
#get the overlap
#get the pts stats

In [15]:
#subgroup of pt with data year not empty
df_notempty_date_year=df1[df1.date_year.notnull()]

df_notempty_grouped_year=df_notempty_date_year.groupby(["person_id"]).size().reset_index(name='counts')
list_notempty_pt_year=df_notempty_grouped_year["person_id"].to_list()
print("pts with year not empty:")
print(len(list_notempty_pt_year))


#subgroup of pt with data month not empty
df_notempty_date_month=df1[df1.date_month.notnull()]

df_notempty_grouped_month=df_notempty_date_month.groupby(["person_id"]).size().reset_index(name='counts')
list_notempty_pt_month=df_notempty_grouped_month["person_id"].to_list()

print("pts with month not empty:")
print(len(list_notempty_pt_month))


#subgroup of pt with data day not empty
df_notempty_date_day=df1[df1.date_day.notnull()]

df_notempty_grouped_day=df_notempty_date_day.groupby(["person_id"]).size().reset_index(name='counts')
list_notempty_pt_day=df_notempty_grouped_day["person_id"].to_list()
print("pts with day not empty:")
print(len(list_notempty_pt_day))

pts with year not empty:
2067
pts with month not empty:
2067
pts with day not empty:
2067


# List of pts with both year and month date

In [16]:
pts_with_both_year_month=[x for x in list_notempty_pt_year if x in list_notempty_pt_month]
len(pts_with_both_year_month)

2067

# How many of these have T test results 

In [17]:
pts_have_yearmonth_with_test_list=[x for x in pts_with_both_year_month if x in  list_pt_test]
print(len(pts_have_yearmonth_with_test_list))

1083


In [18]:
#subgroup of pt with data year empty
df_empty_year=df1[df1.date_year.isnull()]

df_empty_year_grouped=df_empty_year.groupby(["person_id"]).size().reset_index(name='counts')
list_empty_year_pt=df_empty_year_grouped["person_id"].to_list()
print("pts with empty year:")
print(len(list_empty_year_pt))

#subgroup of pt with data month empty
df_empty_month=df1[df1.date_month.isnull()]

df_empty_month_grouped=df_empty_month.groupby(["person_id"]).size().reset_index(name='counts')
list_empty_month_pt=df_empty_month_grouped["person_id"].to_list()
print("pts with empty month:")
print(len(list_empty_month_pt))

#subgroup of pt with data month empty
df_empty_day=df1[df1.date_day.isnull()]

df_empty_day_grouped=df_empty_day.groupby(["person_id"]).size().reset_index(name='counts')
list_empty_day_pt=df_empty_day_grouped["person_id"].to_list()
len(list_empty_day_pt)
print("pts with empty day:")
print(len(list_empty_day_pt))

pts with empty year:
656
pts with empty month:
656
pts with empty day:
656


In [19]:
pt_with_missingyear_useless_list=[x for x in list_empty_year_pt if x not in list_notempty_pt_year]

print("pt with year missing for all drugs and hence useless")
print(len(pt_with_missingyear_useless_list))

pt_with_missingmonth_useless_list=[x for x in list_empty_month_pt if x not in list_notempty_pt_month]

print("pt with year missing for all drugs and hence useless")
print(len(pt_with_missingmonth_useless_list)) 

pt with year missing for all drugs and hence useless
53
pt with year missing for all drugs and hence useless
53


# Useless pts due to missing year or month; enssentially no ADT info captured 53

In [20]:
#useless due to missing year or month

a_list=[x for x in pt_with_missingyear_useless_list if x not in pt_with_missingmonth_useless_list]
b_list=[x for x in pt_with_missingmonth_useless_list if x not in pt_with_missingyear_useless_list]
c_list=[x for x in  pt_with_missingyear_useless_list if x in pt_with_missingmonth_useless_list]

pt_with_emptymonthoryear_list=a_list+b_list+c_list
print(len(a_list))
print(len(b_list))
print(len(c_list))
print(len(pt_with_emptymonthoryear_list))

0
0
53
53


In [21]:
pt_with_emptymonthoryear_list[:3]

[300, 2113, 7167]

In [22]:
#overlap_list=[x for x in list_notempty_pt if x not in list_empty_pt]

#len(overlap_list)

In [23]:
df1[:3]

,person_id,dxdate,dx_year,source_from,evidence,hormone_therapy,cancer_drug,matched_text,category,date_year,date_month,date_day
0,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,20.0
1,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0
2,2,2003-01-07,2003,Medtable,LEUPROLIDE (6 MONTH) 45 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0


In [24]:
df1_copy=df1

# Filling the missing day with 1

In [25]:
df1_copy['date_day'].mask(df1_copy.date_day.isnull(), '1', inplace=True)
df1_copy.date_day=df1_copy.date_day.astype(float)

In [26]:
df1_copy[0:3]

,person_id,dxdate,dx_year,source_from,evidence,hormone_therapy,cancer_drug,matched_text,category,date_year,date_month,date_day
0,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,20.0
1,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0
2,2,2003-01-07,2003,Medtable,LEUPROLIDE (6 MONTH) 45 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0


In [27]:
#m = df1.date_day.isnull()
#df1.where(~m,other='1')

In [28]:
df1_copy["full_date"]=pd.to_datetime((df1_copy.date_year*10000 + df1_copy.date_month*100+df1_copy.date_day).apply(str),format='%Y%m%d')
df1_copy["full_date"]

0       2013-05-20
1       2013-05-23
2       2013-05-23
3       2013-12-04
4       2014-04-09
           ...    
12245   2020-11-13
12246   2020-07-29
12247   2020-08-25
12248   2020-10-02
12249          NaT
Name: full_date, Length: 12250, dtype: datetime64[ns]

In [29]:
#get rid of rows where full_dates are NaT
df1_fulldate_notna= df1_copy[df1_copy["full_date"].notna()]
print(shape(df1_copy)[0])
print(shape(df1_fulldate_notna)[0])

12250
11301


In [30]:
df1_fulldate_notna[:3]

,person_id,dxdate,dx_year,source_from,evidence,hormone_therapy,cancer_drug,matched_text,category,date_year,date_month,date_day,full_date
0,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,20.0,2013-05-20
1,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0,2013-05-23
2,2,2003-01-07,2003,Medtable,LEUPROLIDE (6 MONTH) 45 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0,2013-05-23


In [31]:
df3=df1_fulldate_notna[df1_fulldate_notna.hormone_therapy=="ADT"]
len(df3)

11301

In [32]:
#output this table 
df3_grouped=df3.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_with_ADT_fulldate=df3_grouped["person_id"].to_list()
print(len(list_pt_with_ADT_fulldate))

df3.to_csv("sean_big_cohort_cleanup_table_fixbugs_Feb22_approach#3.csv",encoding='utf-8', index=False)

2067


# Pts with only one ADT use: 450 pts

In [33]:
df3_grouped_pt_adtcount=df3.groupby(["person_id"]).size().reset_index(name='counts')
df3_grouped_pt_adtcount
#pick pts with just one ADT

df_one_adt=df3_grouped_pt_adtcount[(df3_grouped_pt_adtcount["counts"] == 1)]
df_one_adt

list_one_adt_pt=df_one_adt["person_id"].to_list()
print(len(list_one_adt_pt))

450


In [34]:
df3[:3]

,person_id,dxdate,dx_year,source_from,evidence,hormone_therapy,cancer_drug,matched_text,category,date_year,date_month,date_day,full_date
0,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,20.0,2013-05-20
1,2,2003-01-07,2003,Medtable,LEUPROLIDE (3 MONTH) 22.5 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0,2013-05-23
2,2,2003-01-07,2003,Medtable,LEUPROLIDE (6 MONTH) 45 MG IM SYRINGE KIT,ADT,Leuprolide,leuprolide,Hormone Therapy,2013.0,5.0,23.0,2013-05-23


In [35]:
df3_grouped=df3.groupby(["person_id"]).size().reset_index(name='counts')
df3_grouped_pt=df3_grouped.person_id.to_list()
print("pt_list")
print(len(df3_grouped_pt))
pt_list=df3_grouped_pt

pt_list
2067


In [36]:
df_pt=df3[(df3["person_id"] ==38)]
each_pt_dates= df_pt['full_date'].to_list()
each_pt_dates

[Timestamp('2015-11-18 00:00:00'),
 Timestamp('2016-02-24 00:00:00'),
 Timestamp('2016-07-18 00:00:00')]

In [37]:
each_pt_dates[0]

Timestamp('2015-11-18 00:00:00')

In [38]:
# define a function that group consecutive dates with a tolerate gap of 7 months

def group_runs(li,tolerance=212.9):
    out = []
    last = li[0]
    for x in li:
        if float(abs(x-last).days) > tolerance:
            yield out
            out = []
        out.append(x)
        last = x
    yield out

In [39]:
list(group_runs(each_pt_dates))[0]

[Timestamp('2015-11-18 00:00:00'),
 Timestamp('2016-02-24 00:00:00'),
 Timestamp('2016-07-18 00:00:00')]

In [40]:
list(group_runs(each_pt_dates))[0][0]

Timestamp('2015-11-18 00:00:00')

In [41]:
list(group_runs(each_pt_dates))[0][-1]

Timestamp('2016-07-18 00:00:00')

In [42]:
#df4_grouped_bydrug=df3.groupby(["cancer_drug","person_id"]).size().reset_index()
#df4_grouped_bydrug

In [43]:
df3_grouped_pt[:5]

[2, 12, 13, 18, 22]

# How many pts have ADT window assginable? And assign them 

In [44]:
result_list=[]
empty_list=[]
only_one_list=[]
two_but_apart_list=[]
threeandmore_but_apart_list=[]

for y in df3_grouped_pt:
#for y in [2,13,38]:  

    #subset the dataframe to each pt
    #get start dates and end dates to lists
    df_pt=df3[(df3["person_id"] == y)]
    
    #sort and get rid of redundancies
    each_pt_info= sorted(df_pt['full_date'].to_list())
    each_pt_dates=list(dict.fromkeys(each_pt_info))

    if len(each_pt_dates)==0:
        adt_start_date="NaN"
        adt_end_date="NaN"
        empty_list.append(y)
        
    if len(each_pt_dates)==1:
        adt_start_date=each_pt_dates[0]
        adt_end_date="NaN"
        only_one_list.append(y)
        
    if len(each_pt_dates)==2:
        adt_start_date=each_pt_dates[0]
        #less than 7 mons
         
        if float(abs(each_pt_dates[1]-each_pt_dates[0]).days)<212.9: 
            adt_start_date=each_pt_dates[0]
            adt_end_date=each_pt_dates[1]
        #larger than 7 mons            
        if float(abs(each_pt_dates[1]-each_pt_dates[0]).days)>=212.9: 
            adt_start_date=each_pt_dates[0]
            adt_end_date="NaN"
            two_but_apart_list.append(y)

    if len(each_pt_dates)>=3:
        
        #Group consecutive dates with a tolerate gap of 7 months
        #list(group_runs(each_pt_dates))[0]

        
        if len(list(group_runs(each_pt_dates))[0])>1:
            adt_start_date=list(group_runs(each_pt_dates))[0][0]
            adt_end_date=list(group_runs(each_pt_dates))[0][-1] 
        
        if len(list(group_runs(each_pt_dates))[0])==1:
            adt_start_date=list(group_runs(each_pt_dates))[0][0]
            adt_end_date="NaN"
            threeandmore_but_apart_list.append(y)            
                     
    #print(y,adt_start_date,adt_end_date)
    result_per_pt=[y,adt_start_date,adt_end_date]
    result_list.append(result_per_pt)

# How many have only one ADT result? 515 pts
# How many have two but 7 months apart? 31 pts  
# How many have three but the first two dates are 7 months apart? 69 pts
# These are all have end dates as NaN: 615 pts

In [45]:
515+31+69

615

In [46]:
print(len(only_one_list))
print(len(two_but_apart_list))
print(len(threeandmore_but_apart_list))

515
31
69


In [47]:
df_adt_results=pd.DataFrame(result_list)
df_adt_results.columns=(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date'])

In [48]:
df_adt_results['Inital_ADT_timewindow'] = df_adt_results['inital_ADT_end_date'] - df_adt_results['inital_ADT_start_date']
df_adt_results['Inital_ADT_time_window_days']=df_adt_results['Inital_ADT_timewindow'].dt.days

In [49]:
df_adt_results[:5]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days
0,2,2013-05-20,2017-02-01,1353 days,1353.0
1,12,2013-05-06,NaT,NaT,NaN
2,13,2010-06-17,2015-03-17,1734 days,1734.0
3,18,2013-11-12,2014-03-13,121 days,121.0
4,22,2010-09-27,2014-08-04,1407 days,1407.0


# Output results: 1452 pts with ADT usewindow not NaN

In [50]:
df_adt_results_notna= df_adt_results[df_adt_results['Inital_ADT_timewindow'].notna()]
df_adt_results_notna.to_csv("Sean_big_1452pts_approach3_cohort_ADT_window_table_Feb24_2021_fixbugs.csv",encoding='utf-8',index=False)

print(shape(df_adt_results)[0])
print(shape(df_adt_results_notna)[0])

2067
1452


# How many have ADT windows and T test results: 855 pts

In [51]:
list_adt_window_pt=df_adt_results_notna["person_id"].to_list()
pts_have_ADTwindow_with_test_list=[x for x in list_adt_window_pt if x in list_pt_test]
print(len(pts_have_ADTwindow_with_test_list))

855


# Compare the computed ADT windows for the current approach (#3) and previous appraoch (#1) for the shared patients 

In [54]:
#find the pt list and adt window #940 in method 1 for the big cohort
df_approach1=pd.read_csv("./data/pca_pt_940_cohort_with_ADT_both_dates.csv", header=0)
df_approach1[:3]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days,index1
0,2,2009-02-26,2017-02-01,2897 days 00:00:00.000000000,2897.0,0
1,12,2013-05-06,NaN,NaN,NaN,1
2,13,2010-06-17,2018-05-17,2891 days 00:00:00.000000000,2891.0,2


In [57]:
df_adt_results_notna[:3]

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_timewindow,Inital_ADT_time_window_days
0,2,2013-05-20,2017-02-01,1353 days,1353.0
2,13,2010-06-17,2015-03-17,1734 days,1734.0
3,18,2013-11-12,2014-03-13,121 days,121.0


In [58]:
df_approach1_notna= df_approach1[df_approach1['Inital_ADT_time_window_days'].notna()]

In [59]:
df_approach1_pt=df_approach1_notna.groupby(["person_id"]).size().reset_index(name='counts')
list_approach1_pt=df_approach1_pt["person_id"].to_list()
print(len(list_approach1_pt))

940


### for this list of pts, get person id, adtstart1,adtend1,window1, adtstart2,adtend2,window2
### person id list: list_approach1_pt
### dataframe approach 1:df_df_approach1_notna
### dataframe approach 2:df_adt_results_notna

In [60]:
result=[]
#previous 1514pt table version 
df_app1=df_approach1_notna
#current sean 1902 version
df_app2=df_adt_results_notna

for y in list_approach1_pt:
#for y in [2,13,38]:  
    df_app1_pt=df_app1[(df_app1["person_id"]== y)]
    df_app2_pt=df_app2[(df_app2["person_id"] == y)]
    
    if(len(df_app1_pt['inital_ADT_start_date'].to_list()))>0: 
        a1=df_app1_pt['inital_ADT_start_date'].to_list()[0]
    else:
        a1="NaN"
    
    if(len(df_app1_pt['inital_ADT_end_date'].to_list()))>0: 
        b1=df_app1_pt['inital_ADT_end_date'].to_list()[0]
    else:
        b1="NaN"

    if(len(df_app1_pt["Inital_ADT_time_window_days"].to_list()))>0: 
        c1=df_app1_pt["Inital_ADT_time_window_days"].to_list()[0]
    else:
        c1="NaN"         
            
    
    if(len(df_app2_pt['inital_ADT_start_date'].to_list()))>0: 
        a2=df_app2_pt['inital_ADT_start_date'].to_list()[0]
    else:
        a2="NaN"
    
    if(len(df_app2_pt['inital_ADT_end_date'].to_list()))>0: 
        b2=df_app2_pt['inital_ADT_end_date'].to_list()[0]
    else:
        b2="NaN"

    if(len(df_app2_pt["Inital_ADT_time_window_days"].to_list()))>0: 
        c2=df_app2_pt["Inital_ADT_time_window_days"].to_list()[0]
    else:
        c2="NaN"                   

    result_pt=[y,a1,b1,c1,a2,b2,c2]
    result.append(result_pt)

In [61]:
df_adt_results_two_appos=pd.DataFrame(result)
df_adt_results_two_appos.columns=(['person_id', 'inital_ADT_start_date_#1', 'inital_ADT_end_date_#1','Inital_ADT_time_window_days_#1', 'inital_ADT_start_date_#3', 'inital_ADT_end_date_#3','Inital_ADT_time_window_days_#3'])

In [ ]:
df_adt_results_two_appos[:5]

# OUTput results

In [62]:
df_adt_results_two_appos.to_csv("./data/Two_approaches_Compare_ADT_windows_table_fixbugs_Feb24_approach#3.csv",encoding='utf-8', index=False)

# How many pts have baseline T test results? 596/1452 pts

In [63]:
#work with two dataframes, df4 (for the adt info) and df_test
df4=df_adt_results_notna
list_adt_pt=df4["person_id"].to_list()

In [64]:
##get the baseline T dates and values
result_baseline_list=[]
pt_list=list_adt_pt

for y in pt_list:
#for y in [125]:

    #print(y)    
    df4_pt=df4[(df4["person_id"] == y)]
    
    if(pd.isnull(df4_pt['inital_ADT_start_date'].iloc[0])):
        a='NaN'
        b='NaN'
        gap="NaN"
        status='NaN'
        tdate='NaN'
        tvalue='NaN'
        N_t_dates='NaN'
        N_t_dates_beforeADT='NaN'
            
    if(pd.notnull(df4_pt['inital_ADT_start_date'].iloc[0])):
        a=df4_pt['inital_ADT_start_date'].to_list()[0]
        b=df4_pt['inital_ADT_end_date'].to_list()[0]
        gap=df4_pt['Inital_ADT_time_window_days'].to_list()[0]

        df_test_pt=df_test[(df_test["person_id"] == y)]    
        each_pt_tdates =df_test_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones before ADT start date, including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x <=a]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_beforeADT=len(cleaned_aaaa) #including the date when adt started
    
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            #status means no baseline TT avail
            status="N"
        #print(tdate)
        
        if(len(start_sorted_bin)>0):
            #apply the closet function
            #tdate=closest(start_sorted_bin,a)   
            df_test_qualify_pt=df_test_pt[df_test_pt.result_date <=a]
            
            if not df_test_qualify_pt.empty:
                tdate=df_test_qualify_pt['result_date'].to_list()[-1]
                tvalue=df_test_qualify_pt['test_results'].to_list()[-1]
                #status means yes there are baseline TT avail
                status="Y"

    #print(y,a,b,gap, tdate,tvalue,status,N_t_dates,N_t_dates_beforeADT)
    result_per_pt=[y,a,b,gap, tdate,tvalue, status, N_t_dates, N_t_dates_beforeADT]
    result_baseline_list.append(result_per_pt)
    #validate 

In [65]:
df_results_baseline=pd.DataFrame(result_baseline_list)
df_results_baseline.columns=(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date', 'inital_ADT_time_window_days', 'Tdate_justbeforeADT', 'Tvalue_justbeforeADT', "baseline avail Status", "N_T_dates","N_T_dates_beforeADT"])

In [66]:
df_results_baseline[:5]

,person_id,inital_ADT_start_date,inital_ADT_end_date,inital_ADT_time_window_days,Tdate_justbeforeADT,Tvalue_justbeforeADT,baseline avail Status,N_T_dates,N_T_dates_beforeADT
0,2,2013-05-20,2017-02-01,1353.0,2013-02-27,10,Y,82,40
1,13,2010-06-17,2015-03-17,1734.0,2009-09-18,23.72,Y,23,2
2,18,2013-11-12,2014-03-13,121.0,NaT,NaN,N,5,0
3,22,2010-09-27,2014-08-04,1407.0,NaT,NaN,N,3,0
4,25,2013-11-12,2014-02-18,98.0,2013-10-15,31.54,Y,12,1


In [67]:
#output this table 
df_results_baseline_avail=df_results_baseline[df_results_baseline["baseline avail Status"]=="Y"]
print(shape(df_results_baseline)[0])
print(shape(df_results_baseline_avail)[0])

df_results_baseline.to_csv("./data/sean_big_cohort_approach#3_pts_with_adt_window_and_baseline_info_fixbugs_Feb23_2021.csv",encoding='utf-8', index=False)
df_results_baseline_avail.to_csv("./data/sean_big_cohort_approach#3_pts_with_adt_window_and_baseline_T_fixbugs_Feb23_2021.csv",encoding='utf-8', index=False)

1452
596


# How many have recovery and non recovery status?

In [68]:
#work with two dataframes, df4 (for the adt info) and df_test
#df4=df_adt_results_notna
#list_adt_pt=df4["person_id"].to_list()

recovery_result_status75_list=[]
for y in list_adt_pt:     

    df4_pt=df4[(df4["person_id"] == y)]

    if len(df4_pt["inital_ADT_end_date"]==0):        
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'
        
    if len(df4_pt["inital_ADT_end_date"]!=0):
        b=df4_pt['inital_ADT_end_date'].to_list()[0]
        
        df_test_pt=df_test[(df_test["person_id"] == y)]    
        each_pt_tdates=df_test_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa) #including on the adt end date
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"
#get the 75 

        if(len(start_sorted_bin)>0):
            dftemp3_pt=df_test[df_test.test_results >=75][df_test.result_date >=b]
            dftemp4_pt=df_test[df_test.test_results <75][df_test.result_date >=b]

            if not dftemp3_pt.empty:
                status ="Y"
                tdate=dftemp3_pt['result_date'].to_list()[0]
                tvalue=dftemp3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if dftemp3_pt.empty:
                status ="N"
                tdate=dftemp4_pt['result_date'].to_list()[-1]
                tvalue=dftemp4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    recovery_status_per_pt=[y,b,tdate, tvalue,status,N_t_dates,N_t_dates_afterADT]
    recovery_result_status75_list.append(recovery_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [69]:
##test test test: non recovery pts count should not be zero

#work with two dataframes, df4 (for the adt info) and df_test
#df4=df_adt_results_notna
#list_adt_pt=df4["person_id"].to_list()
df1=df_adt_results_notna
df2=df_test
result_status_list=[]

for y in list_adt_pt:     

    
    df1_pt=df1[(df1["person_id"] == y)]
        
    if(pd.isnull(df1_pt['inital_ADT_end_date'].iloc[0])):
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    #if(df1_pt.inital_ADT_end_date.values.any()!=""):
    if(pd.notnull(df1_pt['inital_ADT_end_date'].iloc[0])):

        b=df1_pt['inital_ADT_end_date'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"
#get the 75 

        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=75][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <75][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [70]:
df_recovery_result75=pd.DataFrame(result_status_list)
df_recovery_result75.columns=(['person_id', 'inital_ADT_end_date', 'Tdate_rec_norec75', 'Tvalue_rec_norec75', 'Status75',"N_T_dates","N_T_dates_afterADT"])

df_recovery_result75_recovery= df_recovery_result75[df_recovery_result75["Status75"]=="Y"]
df_recovery_result75_nonrecovery= df_recovery_result75[df_recovery_result75["Status75"]=="N"]
df_recovery_result75_NaN= df_recovery_result75[df_recovery_result75["Status75"]=="NaN"]
test_after_adt_zero=df_recovery_result75[df_recovery_result75["N_T_dates_afterADT"]==0]

In [71]:
df_recovery_result75[35:40]

,person_id,inital_ADT_end_date,Tdate_rec_norec75,Tvalue_rec_norec75,Status75,N_T_dates,N_T_dates_afterADT
35,907,2020-04-12,NaT,NaN,NaN,0,0
36,955,2020-01-15,NaT,NaN,NaN,1,0
37,956,2018-03-29,NaT,NaN,NaN,0,0
38,1178,2020-10-05,NaT,NaN,NaN,0,0
39,1229,2012-08-01,NaT,NaN,NaN,3,0


In [72]:
df_recovery_result75[:5]

print(shape(df_recovery_result75)[0])
print(shape(df_recovery_result75_recovery)[0])
print(shape(df_recovery_result75_nonrecovery)[0])
print(shape(df_recovery_result75_NaN)[0])
print(shape(test_after_adt_zero)[0])

1452
278
260
914
914


# use T value >=300 as a critrion

In [73]:
##test test test: non recovery pts count should not be zero

#work with two dataframes, df4 (for the adt info) and df_test
#df4=df_adt_results_notna
#list_adt_pt=df4["person_id"].to_list()
df1=df_adt_results_notna
df2=df_test
result_status300_list=[]

for y in list_adt_pt:     

    
    df1_pt=df1[(df1["person_id"] == y)]
        
    if(pd.isnull(df1_pt['inital_ADT_end_date'].iloc[0])):
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    #if(df1_pt.inital_ADT_end_date.values.any()!=""):
    if(pd.notnull(df1_pt['inital_ADT_end_date'].iloc[0])):

        b=df1_pt['inital_ADT_end_date'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"
#get the 75 
#get the 300
        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=300][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <300][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status300_list.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [74]:
df_recovery_result300=pd.DataFrame(result_status300_list)
df_recovery_result300.columns=(['person_id', 'inital_ADT_end_date', 'Tdate_rec_norec300', 'Tvalue_rec_norec300', 'Status300',"N_T_dates","N_T_dates_afterADT"])

df_recovery_result300_recovery= df_recovery_result300[df_recovery_result300["Status300"]=="Y"]
df_recovery_result300_nonrecovery= df_recovery_result300[df_recovery_result300["Status300"]=="N"]

In [75]:
df_recovery_result300=pd.DataFrame(result_status300_list)
df_recovery_result300.columns=(['person_id', 'inital_ADT_end_date', 'Tdate_rec_norec300', 'Tvalue_rec_norec300', 'Status300',"N_T_dates","N_T_dates_afterADT"])

df_recovery_result300_recovery= df_recovery_result300[df_recovery_result300["Status300"]=="Y"]
df_recovery_result300_nonrecovery= df_recovery_result300[df_recovery_result300["Status300"]=="N"]

df_recovery_result300[:5]

print(shape(df_recovery_result300)[0])
print(shape(df_recovery_result300_recovery)[0])
print(shape(df_recovery_result300_nonrecovery)[0])

1452
173
365


In [76]:
df_recovery_result300[:5]

print(shape(df_recovery_result300)[0])
print(shape(df_recovery_result300_recovery)[0])
print(shape(df_recovery_result300_nonrecovery)[0])

1452
173
365


In [77]:
173+365

538

# For pts with T baseline, how many recovery how many non recovery? #df_results_baseline

# T recovery threshold = 75.0 ng/dL

In [78]:
#work with two dataframes, df4 (for the adt info) and df_test
#df4=df_adt_results_notna
#list_adt_pt=df4["person_id"].to_list()
df1=df_results_baseline_avail
df2=df_test
result_status_list_baseline75=[]

list_adt_pt_baselineavail=df_results_baseline_avail['person_id'].to_list()

for y in list_adt_pt_baselineavail:     
    df1_pt=df1[(df1["person_id"] == y)]

    if(pd.isnull(df1_pt['inital_ADT_end_date'].iloc[0])):
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    #if(df1_pt.inital_ADT_end_date.values.any()!=""):
    if(pd.notnull(df1_pt['inital_ADT_end_date'].iloc[0])):

        b=df1_pt['inital_ADT_end_date'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"

#get the 75
        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=75][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <75][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list_baseline75.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [80]:
dfbaseline_recovery_result75=pd.DataFrame(result_status_list_baseline75)
dfbaseline_recovery_result75.columns=(['person_id', 'inital_ADT_end_date', 'Tdate_rec_norec75', 'Tvalue_rec_norec75', 'Status75',"N_T_dates","N_T_dates_afterADT"])

dfbaseline_recovery_result75_recovery=dfbaseline_recovery_result75[dfbaseline_recovery_result75["Status75"]=="Y"]
dfbaseline_recovery_result75_nonrecovery=dfbaseline_recovery_result75[dfbaseline_recovery_result75["Status75"]=="N"]
dfbaseline_recovery_result75_nan=dfbaseline_recovery_result75[dfbaseline_recovery_result75["Status75"]=="NaN"]

dfbaseline_recovery_result75[:5]

print(shape(dfbaseline_recovery_result75)[0])

print(shape(dfbaseline_recovery_result75_recovery)[0])
print(shape(dfbaseline_recovery_result75_nonrecovery)[0])

print(shape(dfbaseline_recovery_result75_nan)[0])

dfbaseline_recovery_result75.to_csv("./data/sean_big_cohort_approach3_pts_with_adt_window_and_baseline_T_and_recovery_status75ngdL_fixbugs_Feb22_2021.csv",encoding='utf-8', index=False)

596
168
167
261


In [81]:
#get the pt list for sql input

recovery75_pt_list=dfbaseline_recovery_result75_recovery['person_id'].to_list()
nonrecovery75_pt_list=dfbaseline_recovery_result75_nonrecovery['person_id'].to_list()

df_recovery75_pt_list= DataFrame(recovery75_pt_list,columns=['pt_id'])
df_nonrecovery75_pt_list= DataFrame(nonrecovery75_pt_list,columns=['pt_id'])

df_recovery75_pt_list.to_csv("rec75_fixbug_approach#3.csv",encoding='utf-8', index=False)
df_nonrecovery75_pt_list.to_csv("nonrec75_fixbug_approach#3.csv",encoding='utf-8', index=False)

# T recovery threshold = 300.0 ng/dL

In [82]:
##test test test: non recovery pts count should not be zero

#work with two dataframes, df4 (for the adt info) and df_test
#df4=df_adt_results_notna
#list_adt_pt=df4["person_id"].to_list()
df1=df_results_baseline_avail
df2=df_test
result_status_list_baseline300=[]

list_adt_pt_baselineavail=df_results_baseline_avail['person_id'].to_list()

for y in list_adt_pt_baselineavail:     
    df1_pt=df1[(df1["person_id"] == y)]

    if(pd.isnull(df1_pt['inital_ADT_end_date'].iloc[0])):
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    #if(df1_pt.inital_ADT_end_date.values.any()!=""):
    if(pd.notnull(df1_pt['inital_ADT_end_date'].iloc[0])):

        b=df1_pt['inital_ADT_end_date'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"

#get the 300
        if(len(start_sorted_bin)>0):
            df3_pt=df2_pt[df2_pt.test_results >=300][df2_pt.result_date >=b]
            df4_pt=df2_pt[df2_pt.test_results <300][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    #print(y, b, tdate, tvalue,status, N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list_baseline300.append(result_status_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [83]:
dfbaseline_recovery_result300=pd.DataFrame(result_status_list_baseline300)
dfbaseline_recovery_result300.columns=(['person_id', 'inital_ADT_end_date', 'Tdate_rec_norec300', 'Tvalue_rec_norec300', 'Status300',"N_T_dates","N_T_dates_afterADT"])

dfbaseline_recovery_result300_recovery=dfbaseline_recovery_result300[dfbaseline_recovery_result300["Status300"]=="Y"]
dfbaseline_recovery_result300_nonrecovery=dfbaseline_recovery_result300[dfbaseline_recovery_result300["Status300"]=="N"]

dfbaseline_recovery_result300[:5]

print(shape(dfbaseline_recovery_result300)[0])
print(shape(dfbaseline_recovery_result300_recovery)[0])
print(shape(dfbaseline_recovery_result300_nonrecovery)[0])

dfbaseline_recovery_result300.to_csv("sean_big_cohort_approach#3_pts_with_adt_window_and_baseline_T_and_recovery_status300ngdL_fixbugs_Feb23_2021.csv",encoding='utf-8', index=False)

596
106
229


In [86]:
#get the pt list for sql input
recovery300_pt_list=dfbaseline_recovery_result300_recovery['person_id'].to_list()
nonrecovery300_pt_list=dfbaseline_recovery_result300_nonrecovery['person_id'].to_list()

df_recovery300_pt_list= DataFrame(recovery300_pt_list,columns=['pt_id'])
df_nonrecovery300_pt_list= DataFrame(nonrecovery300_pt_list,columns=['pt_id'])

df_recovery300_pt_list.to_csv("rec300_fixbugs_approach#3.csv",encoding='utf-8', index=False)
df_nonrecovery300_pt_list.to_csv("nonrec300_fixbugs_approach#3.csv",encoding='utf-8', index=False)

In [87]:
nonrecovery300_pt_list[:3]

[2, 13, 25]

In [88]:
dfbaseline_recovery_result300.to_csv("./data/sean_big_cohort_approach#3_pts_with_adt_window_and_baseline_T_and_recovery_status300ngdL_fixbugs_Feb23_5PM_2021.csv",encoding='utf-8', index=False)
df_results_baseline_avail.to_csv("./data/sean_big_cohort_approach#3_pts_with_adt_window_and_baseline_T_fixbugs_Feb23_5PM_2021.csv",encoding='utf-8', index=False)

In [89]:
dfbaseline_recovery_result300

,person_id,inital_ADT_end_date,Tdate_rec_norec300,Tvalue_rec_norec300,Status300,N_T_dates,N_T_dates_afterADT
0,2,2017-02-01,2017-04-11,4.8,N,82,5
1,13,2015-03-17,2017-12-14,4.33,N,23,15
2,25,2014-02-18,2020-01-21,238.49,N,12,11
3,38,2016-07-18,2018-04-16,317.16,Y,20,13
4,53,2016-10-03,NaT,NaN,NaN,26,0
...,...,...,...,...,...,...,...
591,197945,2020-09-09,NaT,NaN,NaN,1,0
592,198103,2020-09-21,NaT,NaN,NaN,2,0
593,201026,2020-10-01,NaT,NaN,NaN,1,0
594,201343,2020-09-01,2020-09-15,4.33,N,3,1


# How many ADT to T is not zero

In [90]:
dfbaseline_recovery_result300['ADT_to_T'] =dfbaseline_recovery_result300['Tdate_rec_norec300'] - dfbaseline_recovery_result300['inital_ADT_end_date']
dfbaseline_recovery_result300['ADT_to_T_days']=dfbaseline_recovery_result300['ADT_to_T'].dt.days

dfbaseline_result300_valid=dfbaseline_recovery_result300[dfbaseline_recovery_result300["Status300"]!="NaN"]
dfbaseline_result300_valid_adt_t_notzero=dfbaseline_result300_valid[dfbaseline_result300_valid['ADT_to_T_days']!=0]
print(len(dfbaseline_result300_valid_adt_t_notzero))

279


In [91]:
dfbaseline_recovery_result300[:3]

,person_id,inital_ADT_end_date,Tdate_rec_norec300,Tvalue_rec_norec300,Status300,N_T_dates,N_T_dates_afterADT,ADT_to_T,ADT_to_T_days
0,2,2017-02-01,2017-04-11,4.8,N,82,5,69 days,69.0
1,13,2015-03-17,2017-12-14,4.33,N,23,15,1003 days,1003.0
2,25,2014-02-18,2020-01-21,238.49,N,12,11,2163 days,2163.0


In [93]:
dfbaseline_result300_valid_adt_t_notzero.to_csv("./data/sean_big_cohort_approach#3_pts_ADT_T_notzero_ptlist.csv",encoding='utf-8', index=False)